# 🐾 AnimalCLEF2025 Competition: Official Starter notebook

The **Goal of the** [AnimalCLEF2025](https://www.kaggle.com/competitions/animal-clef-2025/) competition is to identify individual animal (lynxes, salamanders and sea turtles) in photos. This notebook visualize the provided dataset and propose a baseline solution, based on the state-of-the-art re-identification model [MegaDescriptor](https://huggingface.co/BVRA/MegaDescriptor-L-384). The dataset is split into the database and query sets. For each image from the query set, the goal is to:

- Predict whether the depicted individual is in the database.
- If no, the prediction is `new_individual`.
- If yes, the prediction should be the same as the individual in the database.

## Compute BAKS and BAUS

In [1]:
import numpy as np
from typing import List, Union

def baks_compute(
        y_true: Union[List, np.ndarray],
        y_pred: Union[List, np.ndarray],
        identity_test_only: Union[List, np.ndarray]
    ) -> float:
    """Computes BAKS (balanced accuracy on known samples).
    
    Focuses only on samples with known identities (not in identity_test_only).
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        identity_test_only: Labels of unknown identities (only in test set)
        
    Returns:
        Balanced accuracy score for known samples
    """
    # Convert inputs to numpy arrays with object dtype to handle mixed types
    y_true = np.array(y_true, dtype=object)
    y_pred = np.array(y_pred, dtype=object)
    identity_test_only = np.array(identity_test_only, dtype=object)
    
    # Filter out unknown samples
    mask = ~np.isin(y_true, identity_test_only)
    y_true_known = y_true[mask]
    y_pred_known = y_pred[mask]
    
    if len(y_true_known) == 0:
        return 0.0
    
    # Get unique classes in the filtered true labels
    unique_classes = np.unique(y_true_known)
    n_classes = len(unique_classes)
    
    # Compute per-class accuracy and average
    class_accuracies = []
    for cls in unique_classes:
        cls_mask = (y_true_known == cls)
        if np.sum(cls_mask) > 0:
            cls_acc = np.mean(y_pred_known[cls_mask] == cls)
            class_accuracies.append(cls_acc)
    
    # Return the balanced accuracy (mean of per-class accuracies)
    return np.mean(class_accuracies) if class_accuracies else 0.0

def baus_compute(
        y_true: Union[List, np.ndarray],
        y_pred: Union[List, np.ndarray],
        identity_test_only: Union[List, np.ndarray],
        new_class: Union[int, str]
    ) -> float:
    """Computes BAUS (balanced accuracy on unknown samples).
    
    Focuses only on samples with unknown identities (in identity_test_only).
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        identity_test_only: Labels of unknown identities (only in test set)
        new_class: Label used for identifying unknown samples
        
    Returns:
        Balanced accuracy score for unknown samples
    """
    # Convert inputs to numpy arrays with object dtype to handle mixed types
    y_true = np.array(y_true, dtype=object)
    y_pred = np.array(y_pred, dtype=object)
    identity_test_only = np.array(identity_test_only, dtype=object)
    
    # Filter to include only unknown samples
    mask = np.isin(y_true, identity_test_only)
    y_true_unknown = y_true[mask]
    y_pred_unknown = y_pred[mask]
    
    if len(y_true_unknown) == 0:
        return 0.0
    
    # Get unique unknown classes
    unique_unknown_classes = np.unique(y_true_unknown)
    
    # For each unknown class, check if they were correctly predicted as new_class
    class_accuracies = []
    for cls in unique_unknown_classes:
        cls_mask = (y_true_unknown == cls)
        if np.sum(cls_mask) > 0:
            # For unknown samples, correct prediction is new_class
            cls_acc = np.mean(y_pred_unknown[cls_mask] == new_class)
            class_accuracies.append(cls_acc)
    
    # Return the balanced accuracy (mean of per-class accuracies)
    return np.mean(class_accuracies) if class_accuracies else 0.0

## Dependencies instalation
For the competition we provide two Python packages for loading and preprocessing of available datasets ([wildlife-datasets](https://github.com/WildlifeDatasets/wildlife-datasets)) and tools / method for animal re-identification ([wildlife-tools](https://github.com/WildlifeDatasets/wildlife-tools)).

In [2]:
!pip install git+https://github.com/WildlifeDatasets/wildlife-datasets@develop
!pip install git+https://github.com/WildlifeDatasets/wildlife-tools

  Cloning https://github.com/WildlifeDatasets/wildlife-datasets (to revision develop) to /tmp/pip-req-build-wu2tuqgu
  Running command git clone --filter=blob:none --quiet https://github.com/WildlifeDatasets/wildlife-datasets /tmp/pip-req-build-wu2tuqgu
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/WildlifeDatasets/wildlife-datasets to commit 959c6c01a8317ed4f162ebfdf4b5e63faad60228
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wildlife-datasets: filename=wildlife_datasets-1.0.5-py3-none-any.whl size=88207 sha256=a62deba3f21759c6a4a53cf79897bf5834945964a67ac1fdadcc1c51ed939d6b
  Stored in directory: /tmp/pip-ephem-wheel-cache-8vxz53x6/wheels/c5/e9/19/815dc8ac1a073b6769942e201b8776a681537b3da4bc2c60ab
Successfully built wi

## Dependencies import
We load all the required packages and then define the function `create_sample_submission`, which converts provided predictions and a submission file for the competition.

In [3]:
import os
import numpy as np
import pandas as pd
import timm
import torchvision.transforms as T
from wildlife_datasets.datasets import AnimalCLEF2025
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
def create_sample_submission(dataset_query, predictions, file_name='sample_submission.csv'):
    df = pd.DataFrame({
        'image_id': dataset_query.metadata['image_id'],
        'identity': predictions
    })
    df.to_csv(file_name, index=False)

We need to specify the `root`, where the data are stored and then two image transformations. 
1. The first transform only resizes the images and is used for visualization.
2. The second transform also converts it to torch tensor and is used for operations on neural networks.

In [4]:
root = '/kaggle/input/animal-clef-2025'
transform_display = T.Compose([
    T.Resize([384, 384]),
    ])
transform = T.Compose([
    *transform_display.transforms,
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])

### 📊 Visualizing Data

Since `AnimalCLEF2015` is the child class of `datasets.WildlifeDataset` from [wildlife-datasets](https://github.com/WildlifeDatasets/wildlife-datasets/blob/main/wildlife_datasets/datasets/datasets.py), it inherits all its methods and attributes. The following code specify which `transform` we want to use while loading images and that we want to load labels alongside images.

In [5]:
dataset = AnimalCLEF2025(root, transform=transform_display, load_label=True)

📌 **Plotting a sample grid** of the data. We can see that all animals in the testing sets are turtles. The column `dataset` states that all the photos come from the SeaTurtleID2022 dataset.

In [6]:
dataset.plot_grid()
dataset.metadata

image_id            identity  \
0             0  LynxID2025_lynx_37   
1             1  LynxID2025_lynx_37   
2             2  LynxID2025_lynx_49   
3             3                 NaN   
4             4  LynxID2025_lynx_13   
...         ...                 ...   
15204     15204                 NaN   
15205     15205                 NaN   
15206     15206                 NaN   
15207     15207                 NaN   
15208     15208                 NaN   

                                                    path        date  \
0      images/LynxID2025/database/000f9ee1aad063a4485...         NaN   
1      images/LynxID2025/database/0020edb6689e9f78462...         NaN   
2      images/LynxID2025/database/003152e4145b5b69400...         NaN   
3      images/LynxID2025/query/003b89301c7b9f6d18f722...         NaN   
4      images/LynxID2025/database/003c3f82011e9c3f849...         NaN   
...                                                  ...         ...   
15204  images/SeaTurtleID2022/query/images/fecd2dfed0...  2024-06-07   
15205  images/SeaTurtleID2022/query/images/ff1a0c812b...  2023-06-28   
15206  images/SeaTurtleID2022/query/images/ff22f1cfa6...  2024-06-09   
15207  images/SeaTurtleID2022/query/images/ff5d5116d1...  2023-06-21   
15208  images/SeaTurtleID2022/query/images/ff7f522363...  2023-06-13   

      orientation            species     split          dataset  
0           right               lynx  database       LynxID2025  
1            left               lynx  database       LynxID2025  
2            left               lynx  database       LynxID2025  
3            back               lynx     query       LynxID2025  
4           right               lynx  database       LynxID2025  
...           ...                ...       ...              ...  
15204         NaN  loggerhead turtle     query  SeaTurtleID2022  
15205         NaN  loggerhead turtle     query  SeaTurtleID2022  
15206         NaN  loggerhead turtle     query  SeaTurtleID2022  
15207         NaN  loggerhead turtle     query  SeaTurtleID2022  
15208         NaN  loggerhead turtle     query  SeaTurtleID2022  

[15209 rows x 8 columns]

This is confirmed by showing all datasets from which the data is composed of.

In [7]:
dataset.metadata[['dataset', 'split']].value_counts()

dataset           split   
SeaTurtleID2022   database    8729
LynxID2025        database    2957
SalamanderID2025  database    1388
LynxID2025        query        946
SalamanderID2025  query        689
SeaTurtleID2022   query        500
Name: count, dtype: int64

In [8]:
dataset.metadata[dataset.metadata['dataset'] == "SalamanderID2025"]

image_id            identity  \
3903      3903  SalamanderID2025_2   
3904      3904  SalamanderID2025_2   
3905      3905  SalamanderID2025_2   
3906      3906  SalamanderID2025_2   
3907      3907  SalamanderID2025_3   
...        ...                 ...   
5975      5975                 NaN   
5976      5976                 NaN   
5977      5977                 NaN   
5978      5978                 NaN   
5979      5979                 NaN   

                                                   path        date  \
3903  images/SalamanderID2025/database/images/eafd41...  2017-06-03   
3904  images/SalamanderID2025/database/images/e80dc6...  2018-04-16   
3905  images/SalamanderID2025/database/images/836157...  2019-04-29   
3906  images/SalamanderID2025/database/images/8f4b82...  2018-04-16   
3907  images/SalamanderID2025/database/images/56d75a...  2017-07-08   
...                                                 ...         ...   
5975  images/SalamanderID2025/query/images/f10a58d63...  2024-10-15   
5976  images/SalamanderID2025/query/images/46d8af166...  2024-10-22   
5977  images/SalamanderID2025/query/images/f7ca496bf...  2024-10-22   
5978  images/SalamanderID2025/query/images/360912b59...  2024-10-23   
5979  images/SalamanderID2025/query/images/c9e1f76de...  2024-12-22   

     orientation     species     split           dataset  
3903         top         NaN  database  SalamanderID2025  
3904         top         NaN  database  SalamanderID2025  
3905         top         NaN  database  SalamanderID2025  
3906         top         NaN  database  SalamanderID2025  
3907         top         NaN  database  SalamanderID2025  
...          ...         ...       ...               ...  
5975         top  salamander     query  SalamanderID2025  
5976         top  salamander     query  SalamanderID2025  
5977       right  salamander     query  SalamanderID2025  
5978       right  salamander     query  SalamanderID2025  
5979       right  salamander     query  SalamanderID2025  

[2077 rows x 8 columns]

Now, we can plot the data of any dataset. Here we do it for the SeaTurtleID2022. The images are the same as in the query set. The goal for each image from the query set will be to determine its identity from the database (it will always be in the form `SeaTurtleID2022_t???` or to decide that it is a new individual (it is not present in the database).

In [9]:
idx = dataset.metadata['identity'].str.startswith('SeaTurtleID2022')
idx[idx.isnull()] = False
dataset.plot_grid(idx=idx);

## Inference with MegaDescriptor

Instead of training a classifier, we can just use out of the shelf pretrained models - [MegaDescriptor](https://huggingface.co/BVRA/MegaDescriptor-L-384). We use MegaDescriptor to extract features from all images. 

**Note:** _It is highly recommended to use the GPU acceleration._

Here we extract the similarity between the images. The similarity equals to the cosine similarity between the corresponding feature vectors. Since the cosine similarity reflects the angle between the feature vectors, high similarity means that the feature vectors are close to each other and should depict the same individual.

For each query image we extract the closest image as predicted by MegaDescriptor. Their indices are stored in `pred_idx` and the similarity score of the top match in `pred_scores`.

This is sufficient to perform predictions as shown below in `predictions`. However, we also need to predict whether the individual is new. We utilize the simplest idea and predict that the individual is new whenever the similarity score of the top prediction is below a certain threshold which we arbirtarily selected as 0.6. We create a submission file, which scores 30.0% on the public leaderboard.

# GET UNIQUE IDENTITIES

In [10]:
from wildlife_datasets import datasets, splits
dataset = AnimalCLEF2025(root, transform=transform_display, load_label=True)
df = dataset.df 
df = df[df['split'] == 'database']
df.species.value_counts()

species
loggerhead turtle    8729
lynx                 2957
Name: count, dtype: int64

In [11]:
df.fillna('salamander', inplace=True)
df.species.value_counts()

<ipython-input-11-05ec05b7b4b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna('salamander', inplace=True)


species
loggerhead turtle    8729
lynx                 2957
salamander           1388
Name: count, dtype: int64

In [12]:
splitter = splits.OpenSetSplit(0.7, 0.2)
for idx_train, idx_test in splitter.split(df):
    splits.analyze_split(df, idx_train, idx_test)

Split: time-unaware open-set
Samples: train/test/unassigned/total = 8984/4090/0/13074
Classes: train/test/unassigned/total = 926/832/0/1102
Samples: train only/test only        = 270/2832
Classes: train only/test only/joint  = 270/176/656

Fraction of train set     = 68.72%
Fraction of test set only = 21.66%


In [13]:
df_train, df_test = df.loc[idx_train], df.loc[idx_test]
len(df_train)
len(df_test)

4090

In [14]:
df_train

image_id              identity  \
3083       3083    LynxID2025_lynx_00   
2938       2938    LynxID2025_lynx_00   
534         534    LynxID2025_lynx_00   
1113       1113    LynxID2025_lynx_00   
3458       3458    LynxID2025_lynx_00   
...         ...                   ...   
14704     14704  SeaTurtleID2022_t610   
14708     14708  SeaTurtleID2022_t610   
14697     14697  SeaTurtleID2022_t610   
14702     14702  SeaTurtleID2022_t610   
14699     14699  SeaTurtleID2022_t610   

                                                    path        date  \
3083   images/LynxID2025/database/ca193791069939ef893...  salamander   
2938   images/LynxID2025/database/c0946cd20fd6ba61a62...  salamander   
534    images/LynxID2025/database/2346ca29481a60d3880...  salamander   
1113   images/LynxID2025/database/47a04afea9a27f2e998...  salamander   
3458   images/LynxID2025/database/e380653385d8f1fd317...  salamander   
...                                                  ...         ...   
14704  images/SeaTurtleID2022/database/turtles-data/d...  2022-07-08   
14708  images/SeaTurtleID2022/database/turtles-data/d...  2022-07-08   
14697  images/SeaTurtleID2022/database/turtles-data/d...  2022-07-02   
14702  images/SeaTurtleID2022/database/turtles-data/d...  2022-07-08   
14699  images/SeaTurtleID2022/database/turtles-data/d...  2022-07-02   

      orientation            species     split          dataset  
3083        right               lynx  database       LynxID2025  
2938        right               lynx  database       LynxID2025  
534          left               lynx  database       LynxID2025  
1113         left               lynx  database       LynxID2025  
3458         left               lynx  database       LynxID2025  
...           ...                ...       ...              ...  
14704         top  loggerhead turtle  database  SeaTurtleID2022  
14708        left  loggerhead turtle  database  SeaTurtleID2022  
14697       right  loggerhead turtle  database  SeaTurtleID2022  
14702  salamander  loggerhead turtle  database  SeaTurtleID2022  
14699     topleft  loggerhead turtle  database  SeaTurtleID2022  

[8984 rows x 8 columns]

In [15]:
training_dataloader = dataset.get_subset(df_train.index)
val_dataloader = dataset.get_subset(df_test.index)


training_dataloader.transform = T.Compose([
    *transform_display.transforms,
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
training_dataloader.transform

val_dataloader.transform = T.Compose([
    *transform_display.transforms,
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
val_dataloader.transform

Compose(
    Resize(size=[384, 384], interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)

In [16]:
name = 'hf-hub:BVRA/MegaDescriptor-L-384'
device = 'cuda'
model = timm.create_model(name, num_classes=0, pretrained=True)
extractor = DeepFeatures(model, device=device, batch_size=32, num_workers=0)
features_database = extractor(training_dataloader)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

100%|█████████████████████████████████████████████████████████████| 281/281 [14:51<00:00,  3.17s/it]


In [17]:
train_dataset_wildlife = training_dataloader
val_dataset_wildlife = val_dataloader

In [18]:
features_query = extractor(val_dataloader)

100%|█████████████████████████████████████████████████████████████| 128/128 [06:49<00:00,  3.20s/it]


In [19]:
class FeatureWrapper:
    def __init__(self, features):
        self.features = features

In [20]:
database_identities = train_dataset_wildlife.labels_string
unique_identities = np.unique(database_identities)
n_query = len(val_dataset_wildlife)

avg_features_by_identity = {}
for identity in unique_identities:
    identity_indices = np.where(database_identities == identity)[0]
    
    identity_features = features_database[identity_indices]
    
    avg_feature = np.mean(identity_features[0], axis=0)
    
    avg_feature = avg_feature / np.linalg.norm(avg_feature)
    
    avg_features_by_identity[identity] = avg_feature

unique_identities_list = list(avg_features_by_identity.keys())
avg_features_array = np.array([avg_features_by_identity[identity] for identity in unique_identities_list])
avg_features_wrapped = FeatureWrapper(avg_features_array)

similarity = CosineSimilarity()(features_query, avg_features_wrapped)

pred_idx = similarity.argmax(axis=1)
pred_scores = similarity[range(n_query), pred_idx]

predictions = np.array(unique_identities_list)[pred_idx]
new_individual = 'new_individual'
threshold = 0.01  
predictions[pred_scores < threshold] = new_individual

#validation_preds = create_validation_preds(val_datases_wildlife, predictions, file_name='valids.csv')

In [21]:
def compute_geometric_mean(baks, baus):
    return np.sqrt(baks * baus)

In [22]:
val_true_labels = val_dataset_wildlife.labels_string

In [23]:
unseen_ids = []
all_ids = df['identity'].unique()

for i in all_ids:
    if i in val_dataloader.metadata['identity'].tolist():
        if i not in training_dataloader.metadata['identity'].tolist():
            unseen_ids.append(i)

In [24]:
df['identity'].unique()

array(['LynxID2025_lynx_37', 'LynxID2025_lynx_49', 'LynxID2025_lynx_13',
       ..., 'SeaTurtleID2022_t600', 'SeaTurtleID2022_t603',
       'SeaTurtleID2022_t610'], dtype=object)

In [25]:
baks_score = baks_compute(val_true_labels, preds, unseen_ids)
baus_score = baus_compute(val_true_labels, preds, unseen_ids, "new_individual")
geo_mean = compute_geometric_mean(baks_score, baus_score)

print(f"Balanced Accuracy Known Samples (BAKS): {baks_score:.4f}")
print(f"Balanced Accuracy Unknown Samples (BAUS): {baus_score:.4f}")
print(f"Geometric Mean (BAKS & BAUS): {geo_mean:.4f}")

NameError: name 'preds' is not defined

In [ ]:
thresholds = np.arange(0.1, 0.9, 0.05)
results = []

for threshold in thresholds:
    preds = np.array(unique_identities_list)[pred_idx].copy()
    preds[pred_scores < threshold] = 'new_individual'

    baks = baks_compute(val_true_labels, preds, unseen_ids)
    baus = baus_compute(val_true_labels, preds, unseen_ids, 'new_individual')
    geo_mean = compute_geometric_mean(baks, baus)
    results.append((threshold, baks, baus, geo_mean))

    print(f"Threshold: {threshold:.2f} | BAKS: {baks:.4f} | BAUS: {baus:.4f} | GEO_MEAN: {geo_mean:.4f}")

In [ ]:
import matplotlib.pyplot as plt

thresholds, baks_scores, baus_scores, geo = zip(*results)
plt.plot(thresholds, baks_scores, label='BAKS (Known)')
plt.plot(thresholds, baus_scores, label='BAUS (Unknown)')
plt.plot(thresholds, geo, label='Geometrical Mean')
plt.xlabel('Cosine Threshold')
plt.ylabel('Balanced Accuracy')
plt.legend()
plt.show()

# SUBMISSION

In [ ]:
dataset = AnimalCLEF2025(root, transform=transform, load_label=True)
dataset_database = dataset.get_subset(dataset.metadata['split'] == 'database')
dataset_query = dataset.get_subset(dataset.metadata['split'] == 'query')
n_query = len(dataset_query)


#Loading the model
name = 'hf-hub:BVRA/MegaDescriptor-L-384'
device = 'cuda'
model = timm.create_model(name, num_classes=0, pretrained=True)
extractor = DeepFeatures(model, device=device, batch_size=32, num_workers=0)
features_database = extractor(dataset_database)
features_query = extractor(dataset_query)

In [ ]:
database_identities = dataset_database.labels_string
unique_identities = np.unique(database_identities)
n_query = len(dataset_query)

avg_features_by_identity = {}
for identity in unique_identities:
    identity_indices = np.where(database_identities == identity)[0]
    
    identity_features = features_database[identity_indices]
    
    avg_feature = np.mean(identity_features[0], axis=0)
    
    avg_feature = avg_feature / np.linalg.norm(avg_feature)
    
    avg_features_by_identity[identity] = avg_feature

unique_identities_list = list(avg_features_by_identity.keys())
avg_features_array = np.array([avg_features_by_identity[identity] for identity in unique_identities_list])
avg_features_wrapped = FeatureWrapper(avg_features_array)

similarity = CosineSimilarity()(features_query, avg_features_wrapped)

pred_idx = similarity.argmax(axis=1)
pred_scores = similarity[range(n_query), pred_idx]

predictions = np.array(unique_identities_list)[pred_idx]
new_individual = 'new_individual'
threshold = 0.01  
predictions[pred_scores < threshold] = new_individual

create_sample_submission(dataset_query, predictions, file_name='submission_1.csv')